In [13]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import pathlib
import pysam

import warnings
warnings.filterwarnings("ignore")

output_version = "20250318_v0.1"

# Generate metadata

In [14]:
maindir = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis"
outputdir = "/media/hieunguyen/HNSD_mini/outdir"

Vi_runs = ["R6801", "R6829Vi", "R6873"]
Truong_runs = ["R6782", "R6823", "R6829Truong"]

regions = {
    "Vi": "Vi_amplicons.hg19.bed",
    "Truong": "Truong_amplicons.hg19.bed"
}

regions_hg38 = {
    "Vi": "Vi_amplicons.hg38.bed",
    "Truong": "Truong_amplicons.hg38.bed"
}

mode = "directional"
# mode = "non_directional"

# pic = "Truong"
pic = "Vi"

path_to_main_output = os.path.join(outputdir, "targetMethyl_analysis", output_version, f"{pic}_output", mode)
os.system(f"mkdir -p {path_to_main_output}")


print(f"All output are saved at {path_to_main_output}")


All output are saved at /media/hieunguyen/HNSD_mini/outdir/targetMethyl_analysis/20250318_v0.1/Vi_output/directional


In [15]:
all_cov_files = [item for item in pathlib.Path(maindir).glob("target_methylation_*/06*/*.cov")]

metadata_dict = {
    "filename"  : [item.name.split(".no_deduplicated")[0] for item in all_cov_files],
    "Run" : [str(item).split("/")[6].replace("_no_dedup", "").replace("target_methylation_", "") for item in all_cov_files],
    "path": [str(item) for item in all_cov_files]
}

metadata = pd.DataFrame.from_dict(metadata_dict, orient="columns")
metadata["mode"] = metadata["Run"].apply(lambda x: "directional" if "_without_non_directional" in x else "non_directional")
metadata["Run"] = metadata["Run"].apply(lambda x: x.split("_")[0])
metadata["PIC"] = metadata["Run"].apply(lambda x: "Vi" if x in Vi_runs else "Truong")
metadata["bam_path"] = metadata["path"].apply(lambda x: str(x).replace("06_methylation_extract", "05_sorted_bam").replace(".bedGraph.gz.bismark.zero.cov", ".sorted.bam"))
metadata = metadata[(metadata["mode"] == mode) & (metadata["PIC"] == pic)]

##### regiondf for hg19
regiondf = pd.read_csv(regions[pic], sep = "\t", header = None)
regiondf.columns = ["chrom", "start", "end"]
regiondf = regiondf[["chrom", "start", "end"]]
regiondf["region_name"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf["bam_region"] = regiondf[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

##### regiondf for hg38
regiondf_hg38 = pd.read_csv(regions_hg38[pic], sep = ",", header = None)
regiondf_hg38.columns = ["chrom", "start", "end"]
regiondf_hg38 = regiondf_hg38[["chrom", "start", "end"]]
regiondf_hg38["region_name"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"region_{x[0]}_{x[1]}_{x[2]}", axis = 1
)
regiondf_hg38["bam_region"] = regiondf_hg38[["chrom", "start", "end"]].apply(
    lambda x: f"{x[0].replace('chr', '')}:{x[1]}-{x[2]}", axis = 1
)

metadata

,filename,Run,path,mode,PIC,bam_path
18,3-TMH3_S7527-S7727,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
19,4-TMH4_S7528-S7728,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
20,11-TMM3_S7535-S7735,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
21,1-TMH1_S7525-S7725,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
22,10-TMM2_S7534-S7734,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
23,6-TMS1_S7530-S7730,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
24,7-TMS2_S7531-S7731,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
25,2-TMH2_S7526-S7726,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
26,9-TMM1_S7533-S7733,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...
27,8-TMS3_S7532-S7732,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...


# Count on/off target reads

In [16]:
metadata["num_total_reads"] = metadata["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

def count_read_in_region(bam_path, region, chr_mode = False):
    all_reads = []
    bamfile = pysam.AlignmentFile(bam_path, "rb")
    if chr_mode:
        region = f"chr{region}"
    fetched_obj = bamfile.fetch(region = region)
    for read in fetched_obj:
        all_reads.append(read)
    return(len(all_reads))

for region_name in regiondf.region_name.unique():
    bam_region = regiondf[regiondf["region_name"] == region_name]["bam_region"].values[0]
    metadata[region_name] = metadata["bam_path"].apply(lambda x: count_read_in_region(x, bam_region))
    metadata[f"pct_{region_name}"] = metadata[region_name] / metadata["num_total_reads"] * 100
    
metadata.to_excel(os.path.join(path_to_main_output, f"{pic}_read_count_in_region.xlsx"), index = False)


# Process cov file

In [17]:
metadata.head()

,filename,Run,path,mode,PIC,bam_path,num_total_reads,region_chr16_22825587_22825729,pct_region_chr16_22825587_22825729,region_chr16_22825770_22825948,...,region_chr16_22826024_22826152,pct_region_chr16_22826024_22826152,region_chr16_22826130_22826286,pct_region_chr16_22826130_22826286,region_chr17_75369424_75369590,pct_region_chr17_75369424_75369590,region_chr17_75369739_75369887,pct_region_chr17_75369739_75369887,region_chr17_75369604_75369763,pct_region_chr17_75369604_75369763
18,3-TMH3_S7527-S7727,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,457096,0,0.000000,228618,...,228478,49.984686,0,0.000000,0,0.000000,0,0.000000,0,0.000000
19,4-TMH4_S7528-S7728,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,523290,0,0.000000,0,...,523250,99.992356,261634,49.997898,0,0.000000,0,0.000000,0,0.000000
20,11-TMM3_S7535-S7735,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,671108,664906,99.075857,0,...,0,0.000000,0,0.000000,666,0.099239,5518,0.822222,2759,0.411111
21,1-TMH1_S7525-S7725,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,635258,635256,99.999685,0,...,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
22,10-TMM2_S7534-S7734,R6873,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,directional,Vi,/media/hieunguyen/HNHD01/raw_data/targetMethyl...,2740570,930961,33.969612,898768,...,898404,32.781648,0,0.000000,12298,0.448739,65,0.002372,130,0.004744


In [18]:
regiondf.head()

,chrom,start,end,region_name,bam_region
0,chr16,22825587,22825729,region_chr16_22825587_22825729,16:22825587-22825729
1,chr16,22825770,22825948,region_chr16_22825770_22825948,16:22825770-22825948
2,chr16,22825920,22826053,region_chr16_22825920_22826053,16:22825920-22826053
3,chr16,22826024,22826152,region_chr16_22826024_22826152,16:22826024-22826152
4,chr16,22826130,22826286,region_chr16_22826130_22826286,16:22826130-22826286


In [19]:
all_covdf = dict()
for run in metadata.Run.unique():
      for filename in metadata[metadata["Run"] == run]["filename"].unique():
            path_to_save_cov = os.path.join(path_to_main_output, run, filename)
            os.system(f"mkdir -p {path_to_save_cov}")

            covdf = pd.read_csv(metadata[metadata["filename"] == filename]["path"].values[0], header = None, sep = "\t")
            covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
            all_covdf[filename] = covdf
            for region in regiondf.region_name.unique():
                  chrom = int(region.split("_")[1].replace("chr", ""))
                  start = int(region.split("_")[2])
                  end = int(region.split("_")[3])

                  covdf[(covdf["chrom"] == chrom) & 
                        (covdf["start"] >= start) & (covdf["start"] <= end)].to_excel(f"{path_to_save_cov}/{region}.xlsx", index = False)


# Check sample: CONTROL114, R6447

## Check number of reads

In [ ]:
control114_cov = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.bed"
control114_bam = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam"
control114_cov_hg19 = "/media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.cg_cov.hg19.bed"


path_to_save_cov = os.path.join(path_to_main_output, "R6447", "CONTROL114")
os.system(f"mkdir -p {path_to_save_cov}")
os.system(f"mkdir -p {path_to_save_cov}/hg38")
os.system(f"mkdir -p {path_to_save_cov}/hg19")

covdf = pd.read_csv(control114_cov, header = None, sep = "\t")
covdf.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]

covdfhg19 = pd.read_csv(control114_cov_hg19, header = None, sep = "\t")
covdfhg19.columns = ["chrom", "start", "end", "meth_density", "countC", "countT"]
covdfhg19["countC"] = covdf["countC"].values
covdfhg19["countT"] = covdf["countT"].values

for region in regiondf_hg38.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])

        covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].to_excel(f"{path_to_save_cov}/hg38/{region}.xlsx", index = False)
        
        print(covdf[(covdf["chrom"] == chrom) & 
            (covdf["start"] >= start) & (covdf["start"] <= end)].shape)
        
for region in regiondf.region_name.unique():
        chrom = region.split("_")[1]
        start = int(region.split("_")[2])
        end = int(region.split("_")[3])

        covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)].to_excel(f"{path_to_save_cov}/hg19/{region}.xlsx", index = False)
        
        print(covdfhg19[(covdfhg19["chrom"] == chrom) & 
            (covdfhg19["start"] >= start) & (covdfhg19["start"] <= end)].shape)
        
all_covdf["CONTROL114"] = covdfhg19

(16, 6)
(19, 6)
(10, 6)
(14, 6)
(10, 6)
(11, 6)
(17, 6)
(14, 6)
(16, 6)
(19, 6)
(10, 6)
(14, 6)
(10, 6)
(11, 6)
(17, 6)
(14, 6)


In [21]:
metadata_CONTROL114 = pd.DataFrame.from_dict({
    "filename": ["CONTROL114"],
    "Run": ["R6447"],
    "bam_path": [control114_bam],
    "path": [control114_cov]
})

metadata_CONTROL114["num_total_reads"] = metadata_CONTROL114["bam_path"].apply(lambda x: int(pysam.samtools.view("-c", x, catch_stdout=True)))

for region_name in regiondf_hg38.region_name.unique():
    bam_region = regiondf_hg38[regiondf_hg38["region_name"] == region_name]["bam_region"].values[0]
    metadata_CONTROL114[region_name] = metadata_CONTROL114["bam_path"].apply(lambda x: count_read_in_region(x, bam_region, chr_mode = True))
    metadata_CONTROL114[f"pct_{region_name}"] = metadata_CONTROL114[region_name] / metadata_CONTROL114["num_total_reads"] * 100

metadata_CONTROL114.to_excel(os.path.join(path_to_main_output, f"CONTROL114_read_count_in_region.xlsx"), index = False)

[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/hieunguyen/HNHD01/raw_data/targetMethyl_analysis/CONTROL114/65-CONTROL114CT836T_M550-M750.deduplicated.sort.bam.bai
[W::hts_idx_load3] The index file is older than the data file: /media/

## Compare methylation density between 2 samples